In [1]:
import torch

# Check if CUDA (NVIDIA GPU) is available
if torch.cuda.is_available():
    print("CUDA is available! You can use GPU for computation.")
else:
    print("CUDA is not available. You can use CPU for computation.")


CUDA is available! You can use GPU for computation.


c:\Users\User\.conda\envs\DFU2\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
print(torch.__version__)

1.8.1+cu111


In [3]:
import pytorch_lightning as pl
print(pl.__version__)

1.6.4


In [1]:
from src.model import DecoderDenoisingModel
from src.data import SimpleDataModule
import lightning as pl
from lightning.pytorch import seed_everything, Trainer
from lightning.pytorch.tuner import Tuner
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

seed_everything(4, workers=True)
model_class = DecoderDenoisingModel
dm_class = SimpleDataModule



model = model_class(arch="unet", encoder="resnet50", lr=0.0009)
dm = dm_class(root="D:/AKairys/DFUC2022/Examples/decoder-denoising/data/stranger-sections-2-unlabeled-data-patched",
              batch_size=64)
trainer = Trainer(max_epochs=200, default_root_dir = "D:\AKairys\DFUC2022\Examples\decoder-denoising\output",
                  callbacks=[EarlyStopping(monitor="train_loss", mode="min")])
# create tuner object
tuner = Tuner(trainer)
# optimal lr
# tuner.lr_find(model, dm) # lr of 0.0009 was found optimal
# optimal batch size
# tuner.scale_batch_size(model, dm) #batch of 128 was last good, but it did not worked in training
trainer.fit(model=model, datamodule=dm)

Seed set to 4
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 3080') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Loaded 103680 images from D:/AKairys/DFUC2022/Examples/decoder-denoising/data/stranger-sections-2-unlabeled-data-patched


Loading `train_dataloader` to estimate number of stepping batches.
c:\Users\User\.conda\envs\DFU2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.

  | Name | Type | Params
------------------------------
0 | net  | Unet | 32.5 M
------------------------------
9.0 M     Trainable params
23.5 M    Non-trainable params
32.5 M    Total params
130.086   Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

c:\Users\User\.conda\envs\DFU2\lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


Epoch 0:   0%|          | 2/1604 [00:00<06:53,  3.87it/s, v_num=10, lr=0.0009, train_loss=1.640]

c:\Users\User\.conda\envs\DFU2\lib\site-packages\torch\autograd\graph.py:744: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\cudnn\Conv_v8.cpp:919.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Epoch 12:  70%|██████▉   | 1118/1604 [03:56<01:42,  4.73it/s, v_num=10, lr=0.000891, train_loss=0.257, val_loss=0.256]

Model finetuning

In [ ]:
from src.model import DecoderDenoisingModel
from src.data import SimpleDataModule
from lightning.pytorch import seed_everything, Trainer
from lightning.pytorch.tuner import Tuner
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

seed_everything(4, workers=True)
model_class = DecoderDenoisingModel
dm_class = SimpleDataModule



model = model_class(arch="unet", encoder="resnet50", lr=0.0009)
dm = dm_class(root="D:/AKairys/DFUC2022/Examples/decoder-denoising/data/train-patched",
              batch_size=64, num_val=256)
trainer = Trainer(max_epochs=200, default_root_dir = "D:\AKairys\DFUC2022\Examples\decoder-denoising\output",
                  callbacks=[EarlyStopping(monitor="train_loss", mode="min")])
# create tuner object
tuner = Tuner(trainer)
# optimal lr
# tuner.lr_find(model, dm) # lr of 0.0009 was found optimal
# optimal batch size
# tuner.scale_batch_size(model, dm) #batch of 128 was last good, but it did not worked in training
trainer.fit(model=model, datamodule=dm)